In [ ]:
! pip install gtfstk

  Using cached pandas-0.25.3-cp37-cp37m-manylinux1_x86_64.whl (10.4 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray 0.18.2 requires pandas>=1.0, but you have pandas 0.25.3 which is incompatible.
google-colab 1.0.0 requires pandas~=1.1.0; python_version >= "3.0", but you have pandas 0.25.3 which is incompatible.
fbprophet 0.7.1 requires pandas>=1.0.4, but you have pandas 0.25.3 which is incompatible.


In [ ]:
from pathlib import Path

import gtfstk as gt
import pandas as pd

DATA_DIR = Path("/content")
%ls {DATA_DIR}

# Set study date
DATE = "20220118"  # Tuesday

GTFS.zip  sample_data/


In [ ]:
! pip install --upgrade pandas

In [ ]:
feed = gt.read_gtfs(DATA_DIR/"GTFS.zip", dist_units="km")
feed.describe()

,indicator,value
0,agencies,[Delhi Integrated Multi-Modal Transit System L...
1,timezone,Asia/Kolkata
2,start_date,20210901
3,end_date,20221001
4,num_routes,1270
5,num_trips,50355
6,num_stops,4192
7,num_shapes,0
8,sample_date,20210909
9,num_routes_active_on_sample_date,1187


In [ ]:
feed.validate()

,type,message,table,rows
0,warning,Unrecognized column agency_id,fare_attributes,[]
1,warning,Route has no trips,routes,"[29, 43, 76, 84, 95, 133, 146, 160, 205, 206, ..."
3,warning,"Repeated pair (trip_id, departure_time)",stop_times,"[2153146, 2153192, 2153238, 2153284, 2153330, ..."
2,warning,Stop has no stop times,stops,"[1713, 1714, 2408, 2409, 3056, 3057, 3058, 305..."


In [ ]:
feed.trips.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,50315,50316,50317,50318,50319,50320,50321,50322,50323,50324,50325,50326,50327,50328,50329,50330,50331,50332,50333,50334,50335,50336,50337,50338,50339,50340,50341,50342,50343,50344,50345,50346,50347,50348,50349,50350,50351,50352,50353,50354
route_id,0,0,0,0,0,0,0,0,0,0,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1001,1001,1001,1001,1001,1001,1001,1001,1001,1001,1001,1001,1001,1001,1001,1001,1001,1001,1001,...,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9
service_id,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
trip_id,0_20_20,0_20_30,0_20_40,0_20_50,0_21_00,0_21_10,0_21_20,0_21_30,0_21_40,0_21_50,1000_13_50,1000_14_00,1000_14_20,1000_14_50,1000_15_00,1000_15_10,1000_15_50,1000_16_00,1000_16_10,1000_17_00,1000_17_20,1001_07_20,1001_07_40,1001_08_00,1001_10_00,1001_10_10,1001_10_20,1001_10_30,1001_10_50,1001_11_40,1001_12_00,1001_12_30,1001_13_10,1001_13_40,1001_13_50,1001_14_00,1001_14_20,1001_14_30,1001_14_40,1001_14_50,...,9_15_20,9_15_30,9_15_40,9_15_50,9_16_00,9_16_10,9_16_20,9_16_30,9_16_40,9_16_50,9_17_00,9_17_10,9_17_20,9_17_30,9_17_40,9_17_50,9_18_00,9_18_10,9_18_20,9_18_30,9_18_40,9_18_50,9_19_00,9_19_10,9_19_20,9_19_30,9_19_40,9_19_50,9_20_00,9_20_10,9_20_20,9_20_30,9_20_40,9_20_50,9_21_00,9_21_10,9_21_20,9_21_30,9_21_40,9_21_50
shape_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Only show route, trip, and shape columns: slice columns, AKA filter
feed.trips.filter(["route_id", "trip_id", "shape_id"]).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,50315,50316,50317,50318,50319,50320,50321,50322,50323,50324,50325,50326,50327,50328,50329,50330,50331,50332,50333,50334,50335,50336,50337,50338,50339,50340,50341,50342,50343,50344,50345,50346,50347,50348,50349,50350,50351,50352,50353,50354
route_id,0,0,0,0,0,0,0,0,0,0,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1001,1001,1001,1001,1001,1001,1001,1001,1001,1001,1001,1001,1001,1001,1001,1001,1001,1001,1001,...,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9
trip_id,0_20_20,0_20_30,0_20_40,0_20_50,0_21_00,0_21_10,0_21_20,0_21_30,0_21_40,0_21_50,1000_13_50,1000_14_00,1000_14_20,1000_14_50,1000_15_00,1000_15_10,1000_15_50,1000_16_00,1000_16_10,1000_17_00,1000_17_20,1001_07_20,1001_07_40,1001_08_00,1001_10_00,1001_10_10,1001_10_20,1001_10_30,1001_10_50,1001_11_40,1001_12_00,1001_12_30,1001_13_10,1001_13_40,1001_13_50,1001_14_00,1001_14_20,1001_14_30,1001_14_40,1001_14_50,...,9_15_20,9_15_30,9_15_40,9_15_50,9_16_00,9_16_10,9_16_20,9_16_30,9_16_40,9_16_50,9_17_00,9_17_10,9_17_20,9_17_30,9_17_40,9_17_50,9_18_00,9_18_10,9_18_20,9_18_30,9_18_40,9_18_50,9_19_00,9_19_10,9_19_20,9_19_30,9_19_40,9_19_50,9_20_00,9_20_10,9_20_20,9_20_30,9_20_40,9_20_50,9_21_00,9_21_10,9_21_20,9_21_30,9_21_40,9_21_50
shape_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Find all trips of a specific route: slice rows
feed.trips.loc[lambda x: x.route_id == "320"].T

,20132,20133,20134,20135,20136,20137,20138,20139
route_id,320,320,320,320,320,320,320,320
service_id,1,1,1,1,1,1,1,1
trip_id,320_20_00,320_20_10,320_20_40,320_21_00,320_21_10,320_21_20,320_21_30,320_21_40
shape_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Group all trips by route: group
feed.trips.groupby("route_id").get_group("320").T


,20132,20133,20134,20135,20136,20137,20138,20139
route_id,320,320,320,320,320,320,320,320
service_id,1,1,1,1,1,1,1,1
trip_id,320_20_00,320_20_10,320_20_40,320_21_00,320_21_10,320_21_20,320_21_30,320_21_40
shape_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Compute number of trips per route: split, apply, combine
def my_agg(group):
    d = {}
    d["num_trips"] = group.shape[0]
    return pd.Series(d)

# Use method chaining
(
    feed.trips
    # Split
    .groupby("route_id")
    # Apply and combine
    .apply(my_agg)
    .reset_index()
    # Join in route short name from feed.routes
    .merge(feed.routes.filter(["route_id", "route_short_name", "route_long_name", "route_type"]))
    # Sort
    .sort_values("num_trips", ascending=False)
    .T
)

,775,1042,657,265,126,964,414,335,641,299,159,293,47,844,652,1185,892,286,626,911,877,628,115,496,279,534,574,1063,200,33,1014,228,1007,1103,194,16,1173,136,275,826,...,90,488,230,718,519,733,355,372,890,242,356,1016,347,333,560,1183,537,571,377,1044,1132,619,819,1117,423,725,639,1133,1036,545,231,1062,234,380,258,43,907,199,250,546
route_id,600,86,489,1257,1121,785,256,181,473,148,1155,141,1045,667,483,998,714,135,459,733,70,460,1111,337,128,372,41,879,1195,1031,831,1221,825,916,119,1014,987,1132,1267,649,...,1087,329,1223,548,358,561,2,216,712,1234,20,834,192,18,397,996,375,407,220,861,944,451,640,930,264,554,471,945,854,382,1224,878,1227,224,1250,1040,73,1194,1242,383
num_trips,99,99,99,99,99,99,99,99,99,99,98,98,98,98,98,98,98,97,97,97,97,97,97,97,97,97,96,96,96,96,96,96,96,96,96,96,96,96,96,96,...,3,3,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,1,1,1,1,1,1,1,1,1,1
route_short_name,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
route_long_name,214CLUP,221UP,405AUP,221DOWN,727DOWN,GL23DOWN,429CLUP,GL23UP,211DOWN,254DOWN,258UP,717AUP,181ADOWN,405UP,85DOWN,764DOWN,721DOWN,405ADOWN,214CLDOWN,73UP,894CLUP,764UP,534UP,522CLUP,433CLDOWN,211UP,429CLDOWN,473CLDOWN,405DOWN,721UP,85UP,246CLDOWN,929DOWN,778UP,83UP,236UP,RL-77DOWN,827UP,433CLUP,108UP,...,LTDUP,745STLDOWN,127STLDOWN,578EUP,227ASTLUP,102STLDOWN,827LNKSTLUP,167STLDOWN,262LINKSTLUP,157STLDOWN,887STLDOWN,91LINKSTLDOWN,873STLDOWN,840STLUP,101BDOWN,425STLDOWN,836STLDOWN,886STLDOWN,719DOWN,876STLUP,872STLDOWN,770ALTDUP,859STLUP,205LSTLDOWN,917STLUP,825STLUP,182ACLSTLUP,836ESTLUP,836LSTLDOWN,806STLDOWN,883BLTDDOWN,205LNKSTLDOWN,429LSTLUP,101BUP,745DOWN,182ADOWN,829STLDOWN,836LSTLUP,978AUP,465LINKSTLUP
route_type,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3


In [ ]:
# Fill in shape_dist_traveled column of stop_times for later distance calculations.
trip_stats = feed.compute_trip_stats()

feed = feed.append_dist_to_stop_times(trip_stats)
feed.stop_times.T

,1783943,1783944,1783945,1783946,1783947,1783948,1783949,1783950,1783951,1783952,1783953,1783954,1783955,1783956,1783957,1783958,1783959,1783960,1783961,1783962,1783963,1783964,1783965,1783966,1783967,1783968,1783969,1783970,1783971,1783972,1783973,1783974,1783975,1783976,1783977,1783978,1783979,1783980,1783981,1783982,...,1787663,1787664,1787665,1787666,1787667,1787668,1787669,1787670,1787671,1787672,1787673,1787674,1787675,1787676,1787677,1787678,1787679,1787680,1787681,1787682,1787683,1787684,1787685,1787686,1787687,1787688,1787689,1787690,1787691,1787692,1787693,1787694,1787695,1787696,1787697,1787698,1787699,1787700,1787701,1787702
trip_id,0_20_20,0_20_20,0_20_20,0_20_20,0_20_20,0_20_20,0_20_20,0_20_20,0_20_20,0_20_20,0_20_20,0_20_20,0_20_20,0_20_20,0_20_20,0_20_20,0_20_20,0_20_20,0_20_20,0_20_20,0_20_30,0_20_30,0_20_30,0_20_30,0_20_30,0_20_30,0_20_30,0_20_30,0_20_30,0_20_30,0_20_30,0_20_30,0_20_30,0_20_30,0_20_30,0_20_30,0_20_30,0_20_30,0_20_30,0_20_30,...,9_21_50,9_21_50,9_21_50,9_21_50,9_21_50,9_21_50,9_21_50,9_21_50,9_21_50,9_21_50,9_21_50,9_21_50,9_21_50,9_21_50,9_21_50,9_21_50,9_21_50,9_21_50,9_21_50,9_21_50,9_21_50,9_21_50,9_21_50,9_21_50,9_21_50,9_21_50,9_21_50,9_21_50,9_21_50,9_21_50,9_21_50,9_21_50,9_21_50,9_21_50,9_21_50,9_21_50,9_21_50,9_21_50,9_21_50,9_21_50
arrival_time,20:20:00,20:21:08,20:22:02,20:23:10,20:24:24,20:25:38,20:29:33,20:31:44,20:34:10,20:35:10,20:36:19,20:40:15,20:41:19,20:42:54,20:47:15,20:50:52,20:53:45,20:55:53,20:59:09,21:05:02,20:30:00,20:31:08,20:32:02,20:33:10,20:34:24,20:35:38,20:39:33,20:41:44,20:44:10,20:45:10,20:46:19,20:50:15,20:51:19,20:52:54,20:57:15,21:00:52,21:03:45,21:05:53,21:09:09,21:15:02,...,21:50:00,21:53:35,21:54:41,21:57:11,21:57:49,21:59:44,22:00:54,22:02:59,22:04:52,22:06:37,22:13:10,22:15:44,22:18:53,22:22:04,22:30:14,22:31:55,22:32:56,22:37:19,22:38:45,22:41:51,22:46:25,22:50:55,22:52:09,22:54:11,22:55:09,23:00:39,23:04:42,23:05:57,23:06:23,23:10:40,23:12:26,23:13:54,23:14:54,23:16:04,23:18:20,23:18:54,23:21:09,23:21:47,23:22:45,23:24:24
departure_time,20:20:00,20:21:08,20:22:02,20:23:10,20:24:24,20:25:38,20:29:33,20:31:44,20:34:10,20:35:10,20:36:19,20:40:15,20:41:19,20:42:54,20:47:15,20:50:52,20:53:45,20:55:53,20:59:09,21:05:02,20:30:00,20:31:08,20:32:02,20:33:10,20:34:24,20:35:38,20:39:33,20:41:44,20:44:10,20:45:10,20:46:19,20:50:15,20:51:19,20:52:54,20:57:15,21:00:52,21:03:45,21:05:53,21:09:09,21:15:02,...,21:50:00,21:53:35,21:54:41,21:57:11,21:57:49,21:59:44,22:00:54,22:02:59,22:04:52,22:06:37,22:13:10,22:15:44,22:18:53,22:22:04,22:30:14,22:31:55,22:32:56,22:37:19,22:38:45,22:41:51,22:46:25,22:50:55,22:52:09,22:54:11,22:55:09,23:00:39,23:04:42,23:05:57,23:06:23,23:10:40,23:12:26,23:13:54,23:14:54,23:16:04,23:18:20,23:18:54,23:21:09,23:21:47,23:22:45,23:24:24
stop_id,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,...,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,11,12,13,270,271,0,1,2,3
stop_sequence,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,...,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39
shape_dist_traveled,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Trip stats
trip_stats.T

,0,1,2,3,4,5,6,7,8,9,14652,14653,14654,14655,14656,14657,14658,14659,14660,14661,14662,14663,14664,14665,14666,14667,14668,14669,14670,14671,14672,14673,14674,14675,14676,14677,14678,14679,14680,14681,...,50184,50185,50186,50187,50188,50189,50190,50191,50192,50193,50194,50195,50196,50197,50198,50199,50200,50201,50202,50203,50204,50205,50206,50207,50208,50209,50210,50211,50212,50213,50214,50215,50216,50217,50218,50219,50220,50221,50222,50223
trip_id,0_20_20,0_20_30,0_20_40,0_20_50,0_21_00,0_21_10,0_21_20,0_21_30,0_21_40,0_21_50,1_07_00,1_07_10,1_07_20,1_07_30,1_07_40,1_07_50,1_08_00,1_08_10,1_08_30,1_08_40,1_08_50,1_09_00,1_09_10,1_09_30,1_09_40,1_10_00,1_10_10,1_10_50,1_11_00,1_11_10,1_11_20,1_11_30,1_11_40,1_11_50,1_12_00,1_12_10,1_12_20,1_12_30,1_12_40,1_12_50,...,999_12_00,999_12_10,999_12_20,999_12_30,999_12_40,999_12_50,999_13_00,999_13_10,999_13_20,999_13_50,999_14_10,999_14_30,999_14_40,999_14_50,999_15_00,999_15_10,999_15_20,999_15_30,999_15_40,999_15_50,999_16_00,999_16_10,999_16_20,999_16_40,999_16_50,999_17_00,999_17_10,999_17_20,999_17_30,999_17_40,999_17_50,999_18_00,999_18_10,999_18_20,999_18_30,999_18_40,999_18_50,999_19_00,999_19_10,999_20_30
route_id,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999
route_short_name,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
route_type,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
direction_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
shape_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
num_stops,20,20,20,20,20,20,20,20,20,20,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,...,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46
start_time,20:20:00,20:30:00,20:40:00,20:50:00,21:00:00,21:10:00,21:20:00,21:30:00,21:40:00,21:50:00,07:00:00,07:10:00,07:20:00,07:30:00,07:40:00,07:50:00,08:00:00,08:10:00,08:30:00,08:40:00,08:50:00,09:00:00,09:10:00,09:30:00,09:40:00,10:00:00,10:10:00,10:50:00,11:00:00,11:10:00,11:20:00,11:30:00,11:40:00,11:50:00,12:00:00,12:10:00,12:20:00,12:30:00,12:40:00,12:50:00,...,12:00:00,12:10:00,12:20:00,12:30:00,12:40:00,12:50:00,13:00:00,13:10:00,13:20:00,13:50:00,14:10:00,14:30:00,14:40:00,14:50:00,15:00:00,15:10:00,15:20:00,15:30:00,15:40:00,15:50:00,16:00:00,16:10:00,16:20:00,16:40:00,16:50:00,17:00:00,17:10:00,17:20:00,17:30:00,17:40:00,17:50:00,18:00:00,18:10:00,18:20:00,18:30:00,18:40:00,18:50:00,19:00:00,19:10:00,20:30:00
end_time,21:05:02,21:15:02,21:25:02,21:35:02,21:45:02,21:55:02,22:05:02,22:15:02,22:25:02,22:35:02,08:47:42,08:57:42,09:07:42,09:17:42,09:27:42,09:37:42,09:47:42,09:57:42,10:17:42,10:27:42,10:37:42,10:47:42,10:57:42,11:17:42,11:27:42,11:47:42,11:57:42,12:37:42,12:47:42,12:57:42,13:07:42,13:17:42,13:27:42,13:37:42,13:47:42,13:57:42,14:07:42,14:17:42,14:27:42,14:37:42,...,13:51:00,14:01:00,14:1

In [ ]:
# Route stats
route_stats = feed.compute_route_stats(trip_stats, dates=[DATE])
route_stats.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1147,1148,1149,1150,1151,1152,1153,1154,1155,1156,1157,1158,1159,1160,1161,1162,1163,1164,1165,1166,1167,1168,1169,1170,1171,1172,1173,1174,1175,1176,1177,1178,1179,1180,1181,1182,1183,1184,1185,1186
route_id,0,1,10,100,1000,1001,1002,1003,1004,1006,1007,1008,1009,101,1010,1013,1014,1015,1016,1017,1018,1019,102,1020,1021,1022,1023,1024,1025,1026,1027,103,1030,1031,1032,1033,1034,1035,1036,1037,...,96,960,961,962,963,964,965,968,969,97,970,973,974,975,976,977,978,979,98,980,981,982,983,984,985,986,987,988,989,99,990,991,992,993,994,995,996,997,998,999
route_short_name,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
route_type,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
num_trips,10.0,82.0,61.0,75.0,11.0,48.0,20.0,86.0,12.0,79.0,78.0,18.0,7.0,54.0,22.0,4.0,96.0,13.0,92.0,65.0,53.0,72.0,41.0,59.0,60.0,19.0,36.0,30.0,80.0,48.0,68.0,71.0,21.0,96.0,4.0,13.0,72.0,89.0,47.0,79.0,...,16.0,29.0,96.0,87.0,5.0,61.0,6.0,10.0,9.0,8.0,6.0,62.0,6.0,6.0,55.0,91.0,26.0,92.0,25.0,4.0,72.0,14.0,7.0,38.0,4.0,11.0,96.0,7.0,38.0,42.0,17.0,15.0,23.0,16.0,70.0,93.0,2.0,43.0,98.0,74.0
num_trip_starts,10.0,82.0,61.0,75.0,11.0,48.0,20.0,86.0,12.0,79.0,78.0,18.0,7.0,54.0,22.0,4.0,96.0,13.0,92.0,65.0,53.0,72.0,41.0,59.0,60.0,19.0,36.0,30.0,80.0,48.0,68.0,71.0,21.0,96.0,4.0,13.0,72.0,89.0,47.0,79.0,...,16.0,29.0,96.0,87.0,5.0,61.0,6.0,10.0,9.0,8.0,6.0,62.0,6.0,6.0,55.0,91.0,26.0,92.0,25.0,4.0,72.0,14.0,7.0,38.0,4.0,11.0,96.0,7.0,38.0,42.0,17.0,15.0,23.0,16.0,70.0,93.0,2.0,43.0,98.0,74.0
num_trip_ends,10.0,82.0,61.0,75.0,11.0,48.0,20.0,86.0,12.0,79.0,78.0,18.0,7.0,54.0,22.0,4.0,96.0,13.0,92.0,65.0,51.0,72.0,41.0,59.0,60.0,19.0,36.0,30.0,80.0,48.0,62.0,71.0,21.0,96.0,4.0,13.0,72.0,89.0,47.0,78.0,...,16.0,29.0,96.0,87.0,5.0,61.0,6.0,10.0,9.0,8.0,6.0,62.0,6.0,6.0,55.0,91.0,26.0,92.0,25.0,4.0,72.0,14.0,7.0,38.0,4.0,11.0,96.0,7.0,38.0,42.0,17.0,15.0,23.0,16.0,70.0,93.0,2.0,43.0,95.0,74.0
is_loop,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
is_bidirectional,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
start_time,20:20:00,07:00:00,06:00:00,06:00:00,13:50:00,07:20:00,06:00:00,06:00:00,18:20:00,07:20:00,06:00:00,06:10:00,19:40:00,07:30:00,06:00:00,16:50:00,06:00:00,19:20:00,06:00:00,07:10:00,07:30:00,06:00:00,06:10:00,07:10:00,07:40:00,09:20:00,07:30:00,06:00:00,07:30:00,07:50:00,08:40:00,06:10:00,11:50:00,06:00:00,06:50:00,19:10:00,06:30:00,06:00:00,06:50:00,06:00:00,...,06:00:00,06:00:00,06:00:00,06:00:00,11:20:00,06:00:00,06:00:00,19:50:00,19:50:00,10:20:00,21:00:00,06:40:00,20:50:00,12:30:00,07:10:00,06:00:00,08:50:00,06:00:00,06:00:00,21:10:00,06:40:00,07:40:00,20:30:00,06:00:00,21:20:00,20:40:00,06:10:00,06:00:00,07:00:00,06:50:00,06:00:00,06:00:00,06:00:00,06:00:00,06:00:00,06:00:00,20:00:00,06:30:00,06:10:00,06:00:00
end_time,22:35:02,23:27:42,22:17:13,2

In [ ]:
# Join in route long name
route_stats = route_stats.merge(feed.routes.filter(["route_id", "route_long_name"]))

cols = [
    "route_id",
    "route_short_name", 
    "route_long_name", 
    "route_type", 
    "start_time",
    "end_time",
    "num_trips",
    "max_headway",
    "mean_trip_distance",
    "mean_trip_duration",
    "service_speed",
]


In [ ]:
# The most/least frequent routes are
(
    route_stats
    .filter(cols)
    .sort_values("max_headway")
    .T
)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1147,1148,1149,1150,1151,1152,1153,1154,1155,1156,1157,1158,1159,1160,1161,1162,1163,1164,1165,1166,1167,1168,1169,1170,1171,1172,1173,1174,1175,1176,1177,1178,1179,1180,1181,1182,1183,1184,1185,1186
route_id,0,1,10,100,1000,1001,1002,1003,1004,1006,1007,1008,1009,101,1010,1013,1014,1015,1016,1017,1018,1019,102,1020,1021,1022,1023,1024,1025,1026,1027,103,1030,1031,1032,1033,1034,1035,1036,1037,...,96,960,961,962,963,964,965,968,969,97,970,973,974,975,976,977,978,979,98,980,981,982,983,984,985,986,987,988,989,99,990,991,992,993,994,995,996,997,998,999
route_short_name,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
route_long_name,193STLDOWN,125EXTDOWN,61DOWN,153DOWN,783AUP,219STLDOWN,905STLUP,127DOWN,108STLDOWN,142UP,412DOWN,942DOWN,858STLDOWN,175UP,47ASTL2UP,790+876UP,236UP,891STLUP,410CLDOWN,167UP,TMS-PBAGH,720DOWN,858DOWN,992DOWN,260UP,133UP,85AEXTDOWN,822STLUP,929UP,402CLUP,708UP,588DOWN,961AUP,721UP,243STLUP,833LNKSTLUP,911ADOWN,234CLUP,807AUP,848UP,...,979STLUP,772STLDOWN,205DOWN,103DOWN,790ADOWN,741DOWN,806STLUP,134STLDOWN,429STLDOWN,712UP,258STLDOWN,826UP,825LINKSTLDOWN,825EDOWN,930UP,978DOWN,TMS+LAJPAT-AZADPUR,761UP,107STLUP,819STLDOWN,193AUP,91UP,83STLDOWN,175STLUP,174STLDOWN2,156STLDOWN,RL-77DOWN,03STLUP,117UP,838DOWN,522STLDOWN,507STLUP,114ASTLUP,927EXTDOWN,318UP,724UP,425STLDOWN,961EDOWN,764DOWN,781UP
route_type,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
start_time,20:20:00,07:00:00,06:00:00,06:00:00,13:50:00,07:20:00,06:00:00,06:00:00,18:20:00,07:20:00,06:00:00,06:10:00,19:40:00,07:30:00,06:00:00,16:50:00,06:00:00,19:20:00,06:00:00,07:10:00,07:30:00,06:00:00,06:10:00,07:10:00,07:40:00,09:20:00,07:30:00,06:00:00,07:30:00,07:50:00,08:40:00,06:10:00,11:50:00,06:00:00,06:50:00,19:10:00,06:30:00,06:00:00,06:50:00,06:00:00,...,06:00:00,06:00:00,06:00:00,06:00:00,11:20:00,06:00:00,06:00:00,19:50:00,19:50:00,10:20:00,21:00:00,06:40:00,20:50:00,12:30:00,07:10:00,06:00:00,08:50:00,06:00:00,06:00:00,21:10:00,06:40:00,07:40:00,20:30:00,06:00:00,21:20:00,20:40:00,06:10:00,06:00:00,07:00:00,06:50:00,06:00:00,06:00:00,06:00:00,06:00:00,06:00:00,06:00:00,20:00:00,06:30:00,06:10:00,06:00:00
end_time,22:35:02,23:27:42,22:17:13,23:19:34,18:40:50,23:11:17,12:43:40,22:42:02,22:07:05,22:47:49,23:28:09,17:21:52,22:47:46,23:52:27,10:16:45,21:00:41,23:52:54,22:58:08,23:10:02,23:23:06,24:59:30,23:52:20,23:59:56,23:42:40,23:13:49,23:42:59,22:07:08,13:39:08,23:27:20,23:31:27,24:53:16,21:48:59,19:16:56,23:29:01,22:02:28,23:48:09,23:10:24,22:51:14,23:11:44,24:13:11,...,10:23:38,12:33:45,22:52:58,23:39:36,13:39:30,22:29:26,07:18:17,23:49:22,22:34:18,20:57:08,22:32:12,22:54:18,22:43:08,15:26:20,23:12:03,23:07:32,18:01:31,23:25:25,11:24:10,21:50:10,23:26:44,21:46:10,22:35:26,17:34:23,22:19:07,22:42:03,23:26:55,08:57:12,22:47:11,21:41:47,09:36:57,09:44:43,12:02:08,11:58:47,22:49:08,22:52:21,21:43:11,20:33:18,24:28:46,22:21:00
num_trips,10.0,82.0,61.0,75.0,11.0,48.0,20.0,86.0,12.0,79.0,78.0,18.0,7.0,54.0,22.0,4.0,96.0,13.0,92.0,65.0,53.0,72.0,41.0,59.0,60.0,19.0,36.0,30.0,80.0,48.0,68.0,71.0,21.0,96.0,4.0,13.0,72.0,89.0,47.0,79.0,...,16.0,29.0,96.0,87.0,5.0,61.0,6.0,10.0,9.0,8.0,6.0,62.0,6.0,6.0,55.0,91.0,26.0,92.0,25.0,4.0,72.0,14.0,7.0,38.0,4.0,11.0,96.0,7.0,38.0,42.0,17.0,15.0,23.0,16.0,70.0,93.0,2.0,43.0,98.0,74.0
max_headway,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [ ]:
# The shortest/longest routes are
(
    route_stats
    .filter(cols)
    .sort_values("mean_trip_distance")
    .T
)


,0,794,793,792,791,790,789,795,788,786,785,784,783,782,781,787,796,797,798,813,812,811,810,809,808,807,806,805,804,803,802,801,800,799,780,814,779,777,757,756,...,409,408,407,406,386,385,384,383,382,381,380,379,378,377,376,375,374,373,372,387,296,388,390,405,404,403,402,401,400,399,398,397,396,395,394,393,392,391,389,1186
route_id,0,618,617,616,615,614,613,619,612,610,61,609,608,607,606,611,62,620,621,635,634,633,632,631,630,63,629,628,627,626,625,624,623,622,605,636,604,602,583,582,...,251,250,25,249,23,229,228,227,226,225,224,222,221,220,22,219,218,217,216,230,144,231,233,248,247,246,245,244,243,242,241,240,24,239,237,236,235,234,232,999
route_short_name,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
route_long_name,193STLDOWN,764MPSTLUP,878STLUP,943STLDOWN,254UP,144ADOWN,410STLUP,102STLUP,962STLDOWN,872UP,942EUP,311AUP,119UP,307STLDOWN,741STLDOWN,145UP,445STLUP,826LINKSTLUP,104STLUP,172DOWN,161UP,120UP,728ADOWN,179DOWN,193UP,934STLDOWN,175DOWN,262DOWN,33LNKSTLDOWN,112UP,850DOWN,775EUP,961DOWN,114BDOWN2,751DOWN,971LNKSTLUP,874UP,125EXTUP,396UP,112STLUP,...,858ADOWN,154STLDOWN,182AUP,780UP,764MPSTLDOWN,522CLDOWN,720UP,623UP,745UP,780DOWN,101BUP,981UP,774BDOWN,719DOWN,801STLDOWN,543STLUP,905UP,108DOWN,167STLDOWN,101EXTUP,117DOWN,112AUP,885STLDOWN,829STLUP,445AUP,138ADOWN,144STLDOWN,460STLUP2,128DOWN,141STLUP,887UP,740EXTUP,411LNKSTLUP,821STLDOWN,378UP,826LINKSTLDOWN,801UP,942STLUP,119ADOWN,781UP
route_type,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
start_time,20:20:00,06:00:00,06:20:00,16:00:00,06:40:00,06:00:00,06:00:00,06:00:00,06:00:00,07:10:00,10:10:00,06:20:00,06:40:00,06:00:00,18:30:00,06:30:00,06:00:00,06:10:00,06:00:00,06:00:00,06:50:00,06:20:00,06:10:00,06:00:00,06:30:00,19:50:00,06:00:00,17:30:00,08:20:00,07:10:00,06:00:00,06:50:00,06:10:00,10:10:00,07:50:00,06:00:00,06:00:00,06:00:00,06:00:00,06:00:00,...,11:40:00,17:20:00,15:40:00,07:30:00,18:00:00,06:50:00,07:20:00,08:30:00,17:10:00,06:00:00,06:20:00,08:30:00,06:00:00,07:00:00,06:10:00,20:10:00,06:50:00,06:20:00,21:20:00,08:10:00,06:00:00,06:10:00,06:00:00,06:10:00,07:20:00,10:00:00,20:50:00,20:10:00,06:00:00,20:10:00,07:40:00,07:30:00,19:00:00,18:40:00,06:10:00,21:10:00,08:10:00,06:00:00,06:00:00,06:00:00
end_time,22:35:02,13:56:17,10:12:01,23:42:39,24:12:04,23:06:27,10:05:43,08:33:08,12:51:43,22:34:27,16:33:39,22:58:18,23:53:22,08:17:31,23:09:32,22:41:43,09:55:43,09:00:03,09:37:08,23:40:44,26:17:29,23:51:03,21:59:59,23:23:48,24:20:03,22:17:00,24:10:29,22:47:20,23:13:25,23:54:17,23:46:42,23:29:17,20:48:12,18:38:25,23:33:35,07:20:26,23:07:37,23:25:41,23:27:12,10:11:44,...,16:29:29,24:13:11,23:19:51,21:58:22,23:01:00,23:51:05,23:52:44,23:36:24,20:03:02,20:57:10,07:24:34,23:57:59,23:00:44,09:46:59,23:11:01,22:54:26,22:50:55,22:58:26,22:37:34,18:32:58,22:33:18,23:05:02,21:44:27,08:01:31,23:02:28,16:00:18,22:57:39,22:39:53,23:06:01,22:31:20,22:52:20,23:56:22,22:30:29,23:20:45,23:03:34,22:13:00,23:54:56,10:09:11,23:24:39,22:21:00
num_trips,10.0,14.0,9.0,25.0,95.0,62.0,19.0,11.0,15.0,64.0,17.0,90.0,65.0,13.0,19.0,85.0,10.0,7.0,10.0,49.0,50.0,89.0,43.0,93.0,83.0,10.0,65.0,15.0,19.0,65.0,81.0,64.0,23.0,28.0,47.0,6.0,84.0,90.0,69.0,14.0,...,11.0,21.0,27.0,66.0,18.0,92.0,73.0,81.0,3.0,62.0,1.0,71.0,88.0,2.0,93.0,12.0,65.0,94.0,2.0,15.0,51.0,61.0,4.0,5.0,66.0,19.0,5.0,7.0,77.0,11.0,44.0,82.0,10.0,7.0,86.0,3.0,78.0,9.0,52.0,74.0
max_headway,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [ ]:
# The slowest/fastest routes are
(
    route_stats
    .filter(cols)
    .sort_values("service_speed")
    .T
)

,0,794,793,792,791,790,789,795,788,786,785,784,783,782,781,787,796,797,798,813,812,811,810,809,808,807,806,805,804,803,802,801,800,799,780,814,779,777,757,756,...,409,408,407,406,386,385,384,383,382,381,380,379,378,377,376,375,374,373,372,387,296,388,390,405,404,403,402,401,400,399,398,397,396,395,394,393,392,391,389,1186
route_id,0,618,617,616,615,614,613,619,612,610,61,609,608,607,606,611,62,620,621,635,634,633,632,631,630,63,629,628,627,626,625,624,623,622,605,636,604,602,583,582,...,251,250,25,249,23,229,228,227,226,225,224,222,221,220,22,219,218,217,216,230,144,231,233,248,247,246,245,244,243,242,241,240,24,239,237,236,235,234,232,999
route_short_name,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
route_long_name,193STLDOWN,764MPSTLUP,878STLUP,943STLDOWN,254UP,144ADOWN,410STLUP,102STLUP,962STLDOWN,872UP,942EUP,311AUP,119UP,307STLDOWN,741STLDOWN,145UP,445STLUP,826LINKSTLUP,104STLUP,172DOWN,161UP,120UP,728ADOWN,179DOWN,193UP,934STLDOWN,175DOWN,262DOWN,33LNKSTLDOWN,112UP,850DOWN,775EUP,961DOWN,114BDOWN2,751DOWN,971LNKSTLUP,874UP,125EXTUP,396UP,112STLUP,...,858ADOWN,154STLDOWN,182AUP,780UP,764MPSTLDOWN,522CLDOWN,720UP,623UP,745UP,780DOWN,101BUP,981UP,774BDOWN,719DOWN,801STLDOWN,543STLUP,905UP,108DOWN,167STLDOWN,101EXTUP,117DOWN,112AUP,885STLDOWN,829STLUP,445AUP,138ADOWN,144STLDOWN,460STLUP2,128DOWN,141STLUP,887UP,740EXTUP,411LNKSTLUP,821STLDOWN,378UP,826LINKSTLDOWN,801UP,942STLUP,119ADOWN,781UP
route_type,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
start_time,20:20:00,06:00:00,06:20:00,16:00:00,06:40:00,06:00:00,06:00:00,06:00:00,06:00:00,07:10:00,10:10:00,06:20:00,06:40:00,06:00:00,18:30:00,06:30:00,06:00:00,06:10:00,06:00:00,06:00:00,06:50:00,06:20:00,06:10:00,06:00:00,06:30:00,19:50:00,06:00:00,17:30:00,08:20:00,07:10:00,06:00:00,06:50:00,06:10:00,10:10:00,07:50:00,06:00:00,06:00:00,06:00:00,06:00:00,06:00:00,...,11:40:00,17:20:00,15:40:00,07:30:00,18:00:00,06:50:00,07:20:00,08:30:00,17:10:00,06:00:00,06:20:00,08:30:00,06:00:00,07:00:00,06:10:00,20:10:00,06:50:00,06:20:00,21:20:00,08:10:00,06:00:00,06:10:00,06:00:00,06:10:00,07:20:00,10:00:00,20:50:00,20:10:00,06:00:00,20:10:00,07:40:00,07:30:00,19:00:00,18:40:00,06:10:00,21:10:00,08:10:00,06:00:00,06:00:00,06:00:00
end_time,22:35:02,13:56:17,10:12:01,23:42:39,24:12:04,23:06:27,10:05:43,08:33:08,12:51:43,22:34:27,16:33:39,22:58:18,23:53:22,08:17:31,23:09:32,22:41:43,09:55:43,09:00:03,09:37:08,23:40:44,26:17:29,23:51:03,21:59:59,23:23:48,24:20:03,22:17:00,24:10:29,22:47:20,23:13:25,23:54:17,23:46:42,23:29:17,20:48:12,18:38:25,23:33:35,07:20:26,23:07:37,23:25:41,23:27:12,10:11:44,...,16:29:29,24:13:11,23:19:51,21:58:22,23:01:00,23:51:05,23:52:44,23:36:24,20:03:02,20:57:10,07:24:34,23:57:59,23:00:44,09:46:59,23:11:01,22:54:26,22:50:55,22:58:26,22:37:34,18:32:58,22:33:18,23:05:02,21:44:27,08:01:31,23:02:28,16:00:18,22:57:39,22:39:53,23:06:01,22:31:20,22:52:20,23:56:22,22:30:29,23:20:45,23:03:34,22:13:00,23:54:56,10:09:11,23:24:39,22:21:00
num_trips,10.0,14.0,9.0,25.0,95.0,62.0,19.0,11.0,15.0,64.0,17.0,90.0,65.0,13.0,19.0,85.0,10.0,7.0,10.0,49.0,50.0,89.0,43.0,93.0,83.0,10.0,65.0,15.0,19.0,65.0,81.0,64.0,23.0,28.0,47.0,6.0,84.0,90.0,69.0,14.0,...,11.0,21.0,27.0,66.0,18.0,92.0,73.0,81.0,3.0,62.0,1.0,71.0,88.0,2.0,93.0,12.0,65.0,94.0,2.0,15.0,51.0,61.0,4.0,5.0,66.0,19.0,5.0,7.0,77.0,11.0,44.0,82.0,10.0,7.0,86.0,3.0,78.0,9.0,52.0,74.0
max_headway,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [ ]:
# Map our extreme routes
feed.map_routes(route_ids=["1","2","3","4"])